## Install dependencies

In [1]:
try:
    # some dependencies might not be installed (e.g. on Google Colab)
    import fmpy
except:
    # install the required packages
    !pip install fmpy

## Download files

In [2]:
from fmpy.util import download_file

# download a file from an URL
download_file(url='https://github.com/t-sommer/fmi-webapp-tutorial/raw/main/Heater.fmu',
              checksum='fe5beced10f7dcf3c3aa453dafa6f60da0f7a5071ad8b2d650bdc7dcc6237246')

## Display text and images

Add _formatted_ **text**, images and tables using [Markdown](https://www.markdownguide.org/cheat-sheet/):

### Heater

A constant voltage of 10 V is applied to a temperature dependent resistor of 10*(1+(T-20C)/(235+20C)) Ohms, whose losses v^2/r are dissipated via a thermal conductance of 0.1 W/K to ambient temperature 20 degree C. The resistor is assumed to have a thermal capacity of 1 J/K, having ambient temperature at the beginning of the experiment.

#### Parameters

| Name  | Default | Description              |
|-------|--------:|--------------------------|
| TAmb  | 293.15  | Ambient temperature [K]  |
| TDif  | 2       | Error in temperature [K] |


![Heater model](https://github.com/t-sommer/fmi-webapp-tutorial/blob/main/Heater.png?raw=1)

## Get information about the FMU

In [3]:
from fmpy import dump

# print information about the FMU
dump('Heater.fmu')



Model Info

  FMI Version        2.0
  FMI Type           Model Exchange, Co-Simulation
  Model Name         Heater
  Description        Control temperature of a resistor
  Platforms          c-code, linux64, win64
  Continuous States  1
  Event Indicators   4
  Variables          15
  Generation Tool    Dymola Version 2022 (64-bit), 2021-04-14
  Generation Date    2021-07-12T13:37:22Z

Default Experiment

  Stop Time          10.0
  Tolerance          0.0001

Variables (input, output)

  Name               Causality              Start Value  Unit     Description
  TRes               output                              K        Resulting temperature
  u                  input                        false           true => switch open, false => p--n connected
  T                  output                                       Absolute temperature in degree Celsius as output signal


## Simulate the FMU and plot the result

In [18]:
# hint: you can conveniently generat a Jupyter notebook with the
# FMPy GUI (Tools > Create Jupyter Notebook...) or with the command
# fmpy create-jupyter-notebook <MyModel.fmu>

import os
import fmpy
from fmpy import simulate_fmu, plot_result, read_csv

fmpy.plot_library = 'plotly'  # experimental


filename = 'Heater.fmu'

start_values = {
    # variable                           start   unit      description
    'TAmb':                            (293.15, 'K'),    # Ambient temperature
    'TDif':                                 (2, 'K'),    # Error in temperature
    'constantVoltage.V':                   (10, 'V'),    # Value of constant voltage
    'heatCapacitor.C':                      (1, 'J/K'),  # Heat capacity of element (= cp*m)
    'resistor.R':                          (10, 'Ohm'),  # Resistance at temperature T_ref
    'resistor.T_ref':                  (293.15, 'K'),    # Reference temperature
    'resistor.alpha':     (0.00392156862745098, '1/K'),  # Temperature coefficient of resistance (R_actual = R*(1 + alpha*(T_heatPort - T_ref))
    'thermalConductor.G':                 (0.1, 'W/K'),  # Constant thermal conductance of material
}

# read input.csv (if it exists)
input = read_csv("input.csv") if os.path.isfile("input.csv") else None

# define the variables to record
output = [
    'u',     # true => switch open, false => p--n connected
    'TRes',  # Resulting temperature
    'T',     # Absolute temperature in degree Celsius as output signal
]

# simulate the FMU
result = simulate_fmu(filename, start_values=start_values, input=input, output=output, stop_time=50.0)

# plot the simulation results
plot_result(result)

## Upload files to the notebook

In [38]:
try:
    # this only works in Google Colab
    from google.colab import files
    uploaded = files.upload()
except:
    # use this in Jupyter
    from ipywidgets import FileUpload
    from IPython.display import display
    import time
    uploaded = FileUpload(accept='.csv')
    display(uploaded)

    

FileUpload(value={}, accept='.csv', description='Upload')

In [41]:
try:
    from google.colab import files
    # nothing to do on Google Colab
except:
    if uploaded.data:
        # write the file to disk
        with open("input.csv", "w+b") as i:
            i.write(uploaded.data[0])
        print("File saved as 'input.csv'.")
    else:
        print("No file was uploaded!")

File saved as 'input.csv'.


## Download files from the notebook

In [ ]:
# write the result to disk as a CSV
write_csv('Heater_out.csv', result)

try:
    # this only works in Google Colab
    from google.colab import files
    files.download('Heater_out.csv')
except:
    # use this in Jupyter
    from IPython.display import FileLink
    from fmpy import write_csv
    FileLink('Heater_out.csv')

## Show docstring

In [ ]:
help(dump)

## Open documentation

In [ ]:
dump?

## Access the source code

In [ ]:
dump??

## Show progress of long running simulations

In [68]:
import threading
from IPython.display import display
import ipywidgets as widgets
from time import sleep

stop_time = 100

progress = widgets.FloatProgress(value=0, min=0, max=stop_time, description='Simulating...')

# worker function that runs the simulation asynchronously
def work(progress):
    
    global result
    
    # callback function that gets called after every simulation step
    def step_finished(time, recorder):
        sleep(0.01)  # slow down the simulation
        progress.value = time
        return True
    
    result = simulate_fmu('Heater.fmu', stop_time=stop_time, step_finished=step_finished)

# show the progress bar
display(progress)

# start the simulation thread
thread = threading.Thread(target=work, args=(progress,))
thread.start()
thread.join()

# hide the progress bar
progress.layout.display = "none"

# plot the simulation result
plot_result(result)

FloatProgress(value=0.0, description='Simulating...')

In [ ]:
# TODO: deployment w/ Docker, input file, helper function, progress bar?